### Imports

In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2

### Connection (URL, DB) / Create Table / Import

In [ ]:
import requests
from bs4 import BeautifulSoup
import psycopg2


def connect_and_save_data(matchday, team_name, value):
    try:
        connection = psycopg2.connect(
            dbname='PSQL_ADSFS2023Gruppe15',
            user='ADSFS2023Gruppe15',
            password='ADS_FS_2023_G15!?',
            host='localhost',
            port='5432'
        )

        cursor = connection.cursor()

        create_table_query = '''
            CREATE TABLE IF NOT EXISTS bundesliga_duels_stats (
                id SERIAL PRIMARY KEY,
                matchday INTEGER NOT NULL,
                team_name TEXT NOT NULL,
                value NUMERIC NOT NULL
            );
        '''
        cursor.execute(create_table_query)
        connection.commit()

        insert_data_query = '''
            INSERT INTO bundesliga_duels_stats (matchday, team_name, value)
            VALUES (%s, %s, %s);
        '''
        cursor.execute(insert_data_query, (matchday, team_name, value))
        connection.commit()

    except (Exception, psycopg2.Error) as error:
        print('Fehler beim Verbinden mit der PostgreSQL-Datenbank:', error)
    finally:
        # Schließe die Verbindung
        if connection:
            cursor.close()
            connection.close()


def fetch_and_save_duels_stats():
    for matchday in range(1, 35):
        url = f'https://www.sport.de/fussball/deutschland-bundesliga/se45495/2022-2023/ro132754/spieltag/md{matchday}/teamstatistik-zweikaempfe/'
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            team_elements = soup.find_all('td', class_='team-name team-name-list')
            duels_elements = soup.find_all('td', class_='team_stats-zweikaempfe team_stats-zweikaempfe-format')

            for team_element, duels_element in zip(team_elements, duels_elements):
                team_name = team_element.text.strip()
                duels_value = duels_element.text.strip()

                connect_and_save_data(matchday, team_name, duels_value)

        else:
            print(f'Fehler beim Abrufen der Webseite für Spieltag {matchday} und Zweikämpfe')


fetch_and_save_duels_stats()